# Language Modeling using Ngram

In this Exercise, we are going to create a bigram language model and its variation. We will build one model for each of the following type and calculate their perplexity:
- Unigram Model
- Bigram Model
- Bigram Model with Laplace smoothing
- Bigram Model with Interpolation
- Bigram Model with Kneser-ney Interpolation

We will also use NLTK which is a natural language processing library for python to make our lives easier.



In [1]:
# # #download corpus
# # !wget --no-check-certificate https://github.com/ekapolc/nlp_2019/raw/master/HW4/BEST2010.zip
# # !unzip BEST2010.zip
# !curl -L -o BEST2010.zip https://github.com/ekapolc/nlp_2019/raw/master/HW4/BEST2010.zip
# !unzip -o BEST2010.zip

In [2]:
# !wget https://www.dropbox.com/s/jajdlqnp5h0ywvo/tokenized_wiki_sample.csv
# !curl -L -o tokenized_wiki_sample.csv https://www.dropbox.com/s/jajdlqnp5h0ywvo/tokenized_wiki_sample.csv

In [3]:
#First we import necessary library such as math, nltk, bigram, and collections.
import math
import nltk
import io
import random
from random import shuffle
from nltk import bigrams, trigrams
from collections import Counter, defaultdict
random.seed(999)

BEST2010 is a free Thai NLP dataset by NECTEC usually used as a standard benchmark for various NLP tasks including language modeling. It is separated into 4 domains including article, encyclopedia, news, and novel. The data is already  tokenized using '|' as a separator.

For example,

ตาม|ที่|นางประนอม ทองจันทร์| |กับ| |ด.ช.กิตติพงษ์ แหลมผักแว่น| |และ| |ด.ญ.กาญจนา กรองแก้ว| |ป่วย|สงสัย|ติด|เชื้อ|ไข้|ขณะ|นี้|ยัง|ไม่|ดี|ขึ้น|

In [4]:
total_word_count = 0
best2010 = []
with open('BEST2010/news.txt','r',encoding='utf-8') as f:
  for i,line in enumerate(f):
    line=line.strip()[:-1] #remove the trailing |
    total_word_count += len(line.split("|"))
    best2010.append(line)

In [5]:
#For simplicity, we assumes that each line is a sentence.
print (f'Total sentences in BEST2010 news dataset :\t{len(best2010)}')
print (f'Total word counts in BEST2010 news dataset :\t{total_word_count}')

Total sentences in BEST2010 news dataset :	30969
Total word counts in BEST2010 news dataset :	1660190


We separate the input into 2 sets, train and test data with 70:30 ratio

In [6]:
sentences = best2010
# The data is separated to train and test set with 70:30 ratio.
train = sentences[:int(len(sentences)*0.7)]
test = sentences[int(len(sentences)*0.7):]

#Training data
train_word_count =0
for line in train:
    for word in line.split('|'):
        train_word_count+=1
print ('Total sentences in BEST2010 news training dataset :\t'+ str(len(train)))
print ('Total word counts in BEST2010 news training dataset :\t'+ str(train_word_count))

Total sentences in BEST2010 news training dataset :	21678
Total word counts in BEST2010 news training dataset :	1042797


Here we load the data from Wikipedia which is also already tokenized. It will be used for answering questions in MyCourseville.

In [7]:
import pandas as pd
wiki_data = pd.read_csv("tokenized_wiki_sample.csv")

## Data Preprocessing

Before training any language models, the first step we always do is process the data into the format suited for the LM.

For this exercise, we will use NLTK to help process our data.

In [8]:
from nltk.lm.preprocessing import pad_both_ends, flatten
from nltk.lm.vocabulary import Vocabulary
from nltk import ngrams

We begin by "tokenizing" our training set. Note that the data is already tokenized so we can just split it.

In [9]:
tokenized_train = [t.split("|") for t in train] # "tokenize" each sentence

Next we create a vocabulary with the ```Vocabulary``` class from NLTK. It accepts a list of tokens so we flatten our sentences into one long sentence first.







In [10]:
flat_tokens = list(flatten(tokenized_train)) #join all sentences into one long sentence
vocab = Vocabulary(flat_tokens, unk_cutoff=3) #Words with frequency **below** 3 (not exactly 3) will not be considered in our vocab and will be converted to <UNK>.
vocab.update(["<s>", "</s>"])

Then we replace low frequency words and pad each sentence with \<s\> in the front and \</s\> in the back of each sentence.

Now *each* sentence is going to look something like this:
\["\<s\>", "hello", "my", "name", "is", "\<UNK\>", "\</s\>" \]

In [11]:
tokenized_train = [[token if token in vocab else "<UNK>" for token in sentence] for sentence in tokenized_train]
padded_tokenized_train = [list(pad_both_ends(sentence, n=2)) for sentence in tokenized_train]

Finally, we do the same for the test set and the wiki dataset.

In [12]:
tokenized_test = [t.split("|") for t in test]
tokenized_test = [[token if token in vocab else "<UNK>" for token in sentence] for sentence in tokenized_test]
padded_tokenized_test = [list(pad_both_ends(sentence, n=2)) for sentence in tokenized_test]

tokenized_wiki_test = [t.split("|") for t in wiki_data['tokenized'].tolist()]
tokenized_wiki_test = [[token if token in vocab else "<UNK>" for token in sentence] for sentence in tokenized_wiki_test]
padded_tokenized_wiki_test = [list(pad_both_ends(sentence, n=2)) for sentence in tokenized_wiki_test]

# Unigram

In this section, we will demonstrate how to build a unigram language model <br>
**Important note:** <br>
**\<s\>** = sentence start symbol <br>
**\</s\>** = sentence end symbol

# VERY IMPORTANT:
- In this notebook, we will *not* default the unknown token probability to ```1/len(vocab)``` but instead will treat it as a normal word and let the model learn its probability so that we can compare our results to NLTK.
- **Also make sure that the code in this notebook can be executed without any problem. If we find that you used NLTK to answer questions in MyCourseVille and did not finish the assignment, you will receive a grade of 0 for this assignment.**

In [13]:
class UnigramModel():
  def __init__(self, data, vocab):
    self.unigram_count = defaultdict(lambda: 0.0)
    self.word_count = 0
    self.vocab = vocab
    for sentence in data:
      for w in sentence: #[(word1, ), (word2, ), (word3, )...]
        w = w[0]
        if w in self.vocab:
          self.unigram_count[w] +=1.0
        else:
          self.unigram_count["<UNK>"] += 1.0
        self.word_count+=1

  def __getitem__(self, w):
    w = w[0]  #[(word1, ), (word2, ), (word3, )...]
    if w in self.vocab:
      return self.unigram_count[w]/(self.word_count)
    else:
      return self.unigram_count["<UNK>"]/(self.word_count)

In [14]:
train_unigrams = [list(ngrams(sent, n=1)) for sent in padded_tokenized_train] #creating the unigrams by setting n=1
model = UnigramModel(train_unigrams, vocab)

In [15]:
def getLnValue(x):
      return math.log(x)

In [16]:
#problability of 'นายก'
print(getLnValue(model['นายก']))

#for example, problability of 'นายกรัฐมนตรี' which is an unknown word is equal to
print(getLnValue(model['นายกรัฐมนตรี']))

#problability of 'นายก' 'ได้' 'ให้' 'สัมภาษณ์' 'กับ' 'สื่อ'
prob = getLnValue(model['นายก'])+getLnValue(model['ได้'])+ getLnValue(model['ให้'])+getLnValue(model['สัมภาษณ์'])+getLnValue(model['กับ'])+getLnValue(model['สื่อ'])+getLnValue(model['</s>'])
print ('Problability of a sentence', math.exp(prob))

-2.8280048315526183
-2.8280048315526183
Problability of a sentence 1.5741554829931142e-13


# Perplexity

In order to compare language model we need to calculate perplexity. In this task you should write a perplexity calculation code for the unigram model. The result perplexity should be around 420.67 and
345.12 on train and test data.

## TODO #1 Calculate perplexity

In [17]:
def getLnValue(x):
    return math.log(x) # ln(x)

def calculate_sentence_ln_prob(sentence, model):
    ln_prob = 0.0
    for word in sentence:
        ln_prob += getLnValue(model[word])
    return ln_prob

def perplexity(test,model):
    ln_prob_sum = 0.0
    word_count = 0
    for sentence in test:
        ln_prob_sum += calculate_sentence_ln_prob(sentence, model)
        word_count += len(sentence)
    avg_ln_prob = ln_prob_sum/word_count
    return math.exp(-avg_ln_prob)

In [18]:
test_unigrams = [list(ngrams(sent, n=1)) for sent in padded_tokenized_test]

In [19]:
print(perplexity(train_unigrams,model))
print(perplexity(test_unigrams,model))

420.6667499545085
345.11942873411994


## Q1 MCV
Calculate the perplexity of the model on the wiki test set and answer in MyCourseVille

In [20]:
wiki_test_unigrams = [list(ngrams(sent, n=1)) for sent in padded_tokenized_wiki_test]

In [21]:
print(perplexity([list(flatten(wiki_test_unigrams))], model))
print(f"rounded: {round(perplexity([list(flatten(wiki_test_unigrams))], model))}")

419.56889624155025
rounded: 420


# Bigram

Next, you will create a better language model than a unigram (which is not much to compare with). But first, it is very tedious to count every pair of words that occur in our corpus by ourselves. Lucky for us, nltk provides us a simple library which will simplify the process.

In [22]:
#example of nltk usage for bigram
sentence = 'I always search google for an answer .'
padded_sentence = list(pad_both_ends(sentence.split(), n=2))

print('This is how nltk generate bigram.')
for w1,w2 in bigrams(padded_sentence):
    print(w1,w2)
print('\n<s> and </s> are used as a start and end of sentence symbol. respectively.')

This is how nltk generate bigram.
<s> I
I always
always search
search google
google for
for an
an answer
answer .
. </s>

<s> and </s> are used as a start and end of sentence symbol. respectively.


Now, you should be able to implement a bigram model by yourself. Also, you must create a new perplexity calculation for bigram. The result perplexity should be around 56.46 and 85.38 on train and test data.

## TODO #3 Write Bigram Model

In [104]:
class BigramModel():
  def __init__(self, data, vocab):
    """
    Args:
        data (list of list of tuples): list of sentences, a sentence is a list of tuples (w_i, w_{i+1})
    """
    self.bigram_count = defaultdict(lambda: 0)
    self.unigram_count = defaultdict(lambda: 0)
    self.word_count = 0
    self.vocab = vocab
    for sentence in data: # data = list of sentences(list of tuples(w1, w2))
      for w1, w2 in sentence: # sentence = [(w1, w2), (w2, w3), (w3, w4), ... , (w_f-1, w_f)]
        self.bigram_count[(w1, w2)] += 1
        self.unigram_count[w1] += 1
        self.word_count += 1
      # count last word
      self.unigram_count[sentence[-1][-1]] += 1
      self.word_count += 1

  def __getitem__(self, bigram):
    """Probability of bigram.
    Args:
        bigram (tuple): (w1, w2)
    Returns:
        P(w2|w1) = count(w1, w2) / count(w1)
    """
    w1, w2 = bigram
    return self.bigram_count[(w1, w2)] / self.unigram_count[w1]

## TODO #4 Write Perplexity for Bigram Model

Sum perplexity score at a sentence level, instead of word level

In [131]:
def calculate_bigram_sentence_ln_prob(bigram_sentence, model):
    ln_prob = 0.0
    for w1, w2 in bigram_sentence:
        prob = model[(w1, w2)]
        ln_prob += math.log(prob) if prob>0 else float('-inf')
    return ln_prob

def perplexity(bigram_data, model):
    ln_prob_sum = 0.0
    n = 0
    for sentence in bigram_data:
        ln_prob_sum = calculate_bigram_sentence_ln_prob(sentence, model)
        n += len(sentence)
    avg_ln_prob = ln_prob_sum/n
    # return 2**(-avg_ln_prob)
    return math.exp(-avg_ln_prob)

In [132]:
train_bigrams = [list(ngrams(sent, n=2)) for sent in padded_tokenized_train]
test_bigrams = [list(ngrams(sent, n=2)) for sent in padded_tokenized_test]

In [133]:
bigram_model_scratch = BigramModel(train_bigrams, vocab)

In [134]:
print(perplexity([list(flatten(train_bigrams))], bigram_model_scratch))
print(perplexity([list(flatten(test_bigrams))[:17]], bigram_model_scratch))
print(perplexity([list(flatten(test_bigrams))], bigram_model_scratch))

56.45504870219316
23.979856569091968
inf


## Q2 MCV

In [109]:
wiki_test_bigrams = [list(ngrams(sent, n=2)) for sent in padded_tokenized_wiki_test]

In [110]:
print(perplexity([list(flatten(wiki_test_bigrams))],bigram_model_scratch))

inf


# Smoothing

Usually any ngram models have a sparsity problem, which means it does not have every possible ngram of words in the dataset. Smoothing techniques can alleviate this problem. In this section, you will implement three basic smoothing methods laplace smoothing, interpolation for bigram, and Knesey-Ney smoothing.

## TODO #5 write Bigram with Laplace smoothing (Add-One Smoothing)

The result perplexity on training and testing should be:

    370.23, 361.25 for Laplace smoothing

In [148]:
class BigramWithLaplaceSmoothing():

  def __init__(self, data, vocab):
    self.bigram_count = defaultdict(lambda: 0)
    self.unigram_count = defaultdict(lambda: 0)
    self.vocab = vocab
    self.vocab_size = len(vocab)
    self.word_count = 0
    for sentence in data: # data = list of sentences(list of tuples(w1, w2))
      for w1, w2 in sentence: # sentence = [(w1, w2), (w2, w3), (w3, w4), ... , (w_f-1, w_f)]
        self.bigram_count[(w1, w2)] += 1
        self.unigram_count[w1] += 1
        self.word_count += 1
      # Count the last word
      self.unigram_count[sentence[-1][-1]] += 1
      self.word_count += 1

  def __getitem__(self, bigram):
    """Probability of bigram.
    Args:
        bigram (tuple): (w1, w2)
    Returns:
        P(w2|w1) = count(w1, w2)+1 / count(w1)+V
    """
    w1, w2 = bigram
    numerator = self.bigram_count[(w1, w2)] + 1
    denominator = self.unigram_count[w1] + self.vocab_size
    return numerator / denominator


model = BigramWithLaplaceSmoothing(train_bigrams, vocab)
print(perplexity([list(flatten(train_bigrams))],model), f"(={round(perplexity([list(flatten(train_bigrams))],model), 2)})")
print(perplexity([list(flatten(test_bigrams))], model), f"(={round(perplexity([list(flatten(test_bigrams))], model), 2)})")

370.22795716115024 (=370.23)
361.25384919283107 (=361.25)


## Q3 MCV

In [139]:
print(perplexity([list(flatten(wiki_test_bigrams))],model))
print(f"rounded: {round(perplexity([list(flatten(wiki_test_bigrams))],model))}")

737.5315171494517
rounded: 738


## TODO #6 Write Bigram with Interpolation
Set the lambda value as 0.7 for bigram, 0.25 for unigram, and 0.05 for unknown word.

The result perplexity on training and testing should be:

    70.07, 102.67 for Interpolation

In [ ]:
class BigramWithInterpolation():

  def __init__(self, data, vocab, l = 0.7):
    self.unigram_count = defaultdict(lambda: 0.0)
    self.bigram_count = defaultdict(lambda: 0.0)
    self.total_word_count = 0
    self.vocab = vocab
    self.l = l #l for lambda
    for sentence in data:
        for w1, w2 in sentence:
            self.bigram_count[(w1,w2)] += 1.0
            self.unigram_count[w1] += 1.0
            self.total_word_count += 1

        #account of the last word of each sentence
        self.unigram_count[w2] += 1.0
        self.total_word_count += 1

  def __getitem__(self, bigram):
      w1, w2 = bigram
      unigram_prob = self.unigram_count[w2]/self.total_word_count
      bigram_prob = self.bigram_count[(w1,w2)]/self.unigram_count[w1]

      return 0.7*bigram_prob + 0.25*unigram_prob + 0.05*(1/len(self.vocab))

model = BigramWithInterpolation(train_bigrams, vocab)
print(perplexity([list(flatten(train_bigrams))],model))
print(perplexity([list(flatten(test_bigrams))], model))

## Q4 MCV

In [ ]:
print(perplexity([list(flatten(wiki_test_bigrams))],model))

## Language modeling on multiple domains

Sometimes, we do not have enough data to create a language model for a new domain. In that case, we can improvised by combining several models to improve result on the new domain.

In this exercise you will try to merge two language models from news and article domains to create a language model for the encyclopedia domain.

In [ ]:
# create encyclopeida data (test data)
encyclo_data=[]
with open('BEST2010/encyclopedia.txt','r',encoding='utf-8') as f:
    for i,line in enumerate(f):
        encyclo_data.append(line.strip()[:-1])

(news) First, you should try to calculate perplexity of your bigram with interpolation on encyclopedia data. The  perplexity should be around 236.329

In [ ]:
encyclopedia_bigrams = ...

In [ ]:
# 1) news only on "encyclopedia"
print(perplexity([list(flatten(encyclopedia_bigrams))], model))

## TODO #7 - Langauge Modelling on Multiple Domains
Combine news and article datasets to create another bigram model and evaluate it on the encyclopedia data.



(article) For your information, a bigram model with interpolation using article data to test on encyclopedia data has a perplexity of 218.55

In [ ]:
# 2) article only on "encyclopedia"
best2010_article=[]
with open('BEST2010/article.txt','r',encoding='utf-8') as f:
    for i,line in enumerate(f):
        best2010_article.append(line.strip()[:-1])

combined_total_word_count = 0
for line in best2010_article:
    combined_total_word_count += len(line.split('|'))

article_bigrams = ...
article_vocab = ...

model = BigramWithInterpolation(article_bigrams, article_vocab)

In [ ]:
print('Perplexity of the bigram model using article data with interpolation smoothing on encyclopedia test data',perplexity([list(flatten(encyclopedia_bigrams))], model))

In [ ]:
# 3) train on news + article, test on "encyclopedia"
best2010_article_and_news = best2010_article.copy()
with open('BEST2010/news.txt','r',encoding='utf-8') as f:
    for i,line in enumerate(f):
        best2010_article_and_news.append(line.strip()[:-1])

combined_bigrams = ...
combined_vocab = ...

combined_model = BigramWithInterpolation(combined_bigrams, combined_vocab)
print('Perplexity of the combined Bigram model with interpolation smoothing on encyclopedia test data',perplexity([list(flatten(encyclopedia_bigrams))], combined_model))

## Q5 MCV

Did you get a better or worse result when using combined data?

## TODO #8 - Kneser-ney on "News"

<!-- Reimplement equation 4.33 in SLP textbook (https://lagunita.stanford.edu/c4x/Engineering/CS-224N/asset/slp4.pdf) -->

Implement Bigram Knerser-ney LM. The result perplexity should be around 65.81, 92.88 on train and test data. Be careful not to mix up vocab from the above section!


In [ ]:
class BigramKneserNey():

  def __init__(self, data, vocab):
    pass


  def __getitem__(self, x):
    pass

model = BigramKneserNey(train_bigrams, vocab)
print(perplexity([list(flatten(train_bigrams))],model))
print(perplexity([list(flatten(train_bigrams))[:1000]],model))
print(perplexity([list(flatten(test_bigrams))[:1000]], model))
print(perplexity([list(flatten(test_bigrams))], model))

## Q6 MCV

In [ ]:
print(perplexity([list(flatten(wiki_test_bigrams))],model))